In [46]:
import pandas as pd
# Defining sets and parameters

# factories
factories = ["IJmuiden","Segal","South Wales"]

# types of rebars and lengths
rebars = {      
    "A": {"length": 2.4},
    "B": {"length": 3.6},
    "C": {"length": 4.2}
} 

# types of long bars and lengths
longbars = {
    1: {"length": 9},
    2: {"length": 12}
}

diameter=0.057 #in meters
density=7.85 #tons/m^3

# production capacity in tonnes per period
production_capacity = {
    "IJmuiden": 12,   
    "Segal": 10,
    "South Wales": 28
}

# customer areas
customers = ["Bochum", "Boenen", "Dortmund", "Gelsenkirchen", "Hagen", "Iserlohn", "Neuss", "Schwerte"]


# time periods
periods = [1,2,3,4]


# demand for each type of rebar
demand = {}
demand["A"] = {
    "Period": [1, 2, 3, 4],
    "Bochum": [2, 6, 5, 3],
    "Boenen": [4, 8, 5, 10],
    "Dortmund": [2, 7, 6, 5],
    "Gelsenkirchen": [5, 5, 5, 5],
    "Hagen": [19, 23, 25, 16],
    "Iserlohn": [13, 19, 17, 14],
    "Neuss": [20, 16, 14, 26],
    "Schwerte": [4, 5, 3, 4]
}

demand["B"] = {
    "Period": [1, 2, 3, 4],
    "Bochum": [4, 5, 7, 8],
    "Boenen": [5, 8, 12, 13],
    "Dortmund": [4, 5, 8, 10],
    "Gelsenkirchen": [9, 10, 6, 6],
    "Hagen": [15, 33, 31, 33],
    "Iserlohn": [22, 26, 20, 27],
    "Neuss": [12, 23, 30, 30],
    "Schwerte": [2, 8, 2, 6]
}

demand["C"] = {
    "Period": [1, 2, 3, 4],
    "Bochum": [6, 7, 7, 7],
    "Boenen": [6, 10, 15, 12],
    "Dortmund": [7, 6, 4, 12],
    "Gelsenkirchen": [10, 9, 9, 10],
    "Hagen": [12, 35, 33, 38],
    "Iserlohn": [14, 25, 23, 24],
    "Neuss": [22, 32, 31, 31],
    "Schwerte": [5, 6, 7, 2]
}

#demand["B"]["Neuss"][0]

fixed_cost = {
    "IJmuiden": 130,   
    "Segal": 150,
    "South Wales": 100
}

variable_cost = 0.5

distance = {
    "Bochum": {"IJmuiden": 250, "Segal": 203, "South Wales": 866},
    "Boenen": {"IJmuiden": 282, "Segal": 242, "South Wales": 914},
    "Dortmund": {"IJmuiden": 266, "Segal": 222, "South Wales": 885},
    "Gelsenkirchen": {"IJmuiden": 234, "Segal": 198, "South Wales": 859},
    "Hagen": {"IJmuiden": 289, "Segal": 206, "South Wales": 903},
    "Iserlohn": {"IJmuiden": 299, "Segal": 226, "South Wales": 913},
    "Neuss": {"IJmuiden": 259, "Segal": 140, "South Wales": 843},
    "Schwerte": {"IJmuiden": 279, "Segal": 216, "South Wales": 901}
}

# Now you can access distances like this:
print(distance["Bochum"]["Segal"])  # Output: 203

# possible number of long bars
n = 1000
possible_longbars = set(range(1, n + 1))


203


In [48]:
## Import pyomo environment and pyomo optimizer
import pyomo.environ as pe
import pyomo.opt as po
import math

time_period = 1

m = pe.ConcreteModel()
m.x = pe.Var(rebars, longbars, possible_longbars, factories, domain = pe.Integers)
m.y = pe.Var(longbars, possible_longbars, factories, domain = pe.Binary)
m.z = pe.Var(factories, customers, domain = pe.Binary)

obj_expr = sum(fixed_cost[f] * m.z[f,c] + variable_cost * distance[c][f] * sum(rebars[r]["length"] for r in rebars)* diameter/2*math.pi for c in customers for f in factories)
m.obj = pe.Objective(sense = pe.minimize, expr= obj_expr)

# Defining the cuts in long bars
@m.Constraint(longbars, possible_longbars, factories)
def cuts(m, l, n, f):
    return sum(rebars[r]["length"] * m.x[(r,l,n,f)] for r in rebars) <= longbars[l]["length"]*m.y[(l,n,f)]

# Supply constraint
@m.Constraint(factories)
def supply(m, f):
    return sum(m.y[(l,n,f)]*longbars[l]["length"] * diameter/2 * math.pi for n in possible_longbars for l in longbars)  <= production_capacity[f]

# Demand constraint
@m.Constraint(factories, customers, rebars)
def demand(m, f, c, r):
    return sum(m.x[(r,l,n,f)] for n in possible_longbars for l in longbars) == demand[r][c][time_period-1] * m.z[(f,c)]


# Each customer is served by 1 factory only
@m.Constraint(customers)
def one_supplier(m, c):
    return sum(m.z[(f,c)] for f in factories) == 1


solver = po.SolverFactory('gurobi')
result = solver.solve(m, tee = True)

print(result.solver.status)
print(result.solver.termination_condition)

Read LP format model from file /var/folders/dh/_xb_27wd2qg4j5p7s0bnfz1w0000gn/T/tmp_e99y0fn.pyomo.lp
Reading time = 0.16 seconds
x1: 6083 rows, 24025 columns, 174096 nonzeros
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[x86] - Darwin 23.6.0 23H417)

CPU model: Intel(R) Core(TM) i5-8210Y CPU @ 1.60GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 6083 rows, 24025 columns and 174096 nonzeros
Model fingerprint: 0x4429a640
Variable types: 1 continuous, 24024 integer (6024 binary)
Coefficient statistics:
  Matrix range     [8e-01, 2e+01]
  Objective range  [1e+02, 5e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+01]
Presolve removed 0 rows and 1 columns
Presolve time: 0.07s

Explored 0 nodes (0 simplex iterations) in 0.09 seconds (0.07 work units)
Thread count was 1 (of 4 available processors)

Solution count 0

Model is infeasible
Best objective -, best bound -, gap -
model.name="unknown";
    - terminati

In [ ]:
#debug
#model.pprint()
